In [1]:
import os
import shutil

os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras.backend as K
K.set_image_dim_ordering('tf')
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# preprocessing, move files to current project folder use soft link to faster and reduce space usage

keras_download_train = '/home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/train'
keras_download_test = '/home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/test'
train_filenames_list = os.listdir(keras_download_train)

train_cat = filter(lambda x:x[:3] == 'cat', train_filenames_list)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames_list)


def check_make_dir(*dirs):
    print("example input ('folder1','folder2')")
    for count,dirname in enumerate(dirs):
        if os.path.exists(dirname) and os.path.isdir(dirname):
            shutil.rmtree(dirname)
        os.mkdir(dirname)
        
check_make_dir('train2','train2/cat','train2/dog','test2')
        
        
os.symlink(keras_download_test, 'test2/test')

for filename in train_cat:
    os.symlink(keras_download_train+'/'+filename, 'train2/cat/'+filename)

for filename in train_dog:
    os.symlink(keras_download_train+'/'+filename, 'train2/dog/'+filename)
    



example input ('folder1','folder2')


In [5]:

def write_gap(input_model, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    x = Input((height, width, 3))
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = input_model(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    print("stage2")

    #my add starts
    train_filenames = train_generator.filenames
    train_nb_samples = len(train_filenames)

    test_filenames = test_generator.filenames
    test_nb_samples = len(test_filenames)
    
    #my add finish
    
    print("stage3")
    
    train = model.predict_generator(train_generator,steps=train_nb_samples,verbose=1)
    test = model.predict_generator(test_generator,steps=test_nb_samples,verbose=1)
    
    print("stage4")

                                   
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)


write_gap(ResNet50, (224, 224))
'''
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)
'''


Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
stage2
stage3
  659/25000 [..............................] - ETA: 15:13:27

KeyboardInterrupt: 

In [6]:
print("test")

test


In [7]:
K.tensorflow_backend._get_available_gpus()

[]